In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer

global raw_restaurant_dt
# 1. 데이터 로드 및 전처리
def load_and_preprocess(file_path_user, file_paths_restaurant):
    """
    사용자와 식당 데이터를 로드하고 전처리합니다.

    Parameters:
        file_path_user (str): 사용자 데이터 파일 경로.
        file_paths_restaurant (list): 식당 데이터 파일 경로 리스트.

    Returns:
        tuple: 전처리된 사용자 데이터, 식당 데이터, 사용자 ID, 식당 ID, 통합 식당 데이터
    """
    user_data = pd.read_csv(file_path_user)

    # 식당 데이터 통합
    restaurant_data_list = [pd.read_csv(file_path) for file_path in file_paths_restaurant]
    restaurant_data = pd.concat(restaurant_data_list, ignore_index=True)

    # 사용자와 식당의 ID 추출
    user_ids = user_data['nickname'].tolist()
    restaurant_ids = restaurant_data['가게명'].tolist()

    # 숫자형 데이터 추출
    user_data_numeric = user_data.select_dtypes(include=['float64', 'int64'])
    restaurant_data_numeric = restaurant_data.select_dtypes(include=['float64', 'int64'])

    # 결측치 처리
    imputer = SimpleImputer(strategy='mean')
    user_data_imputed = imputer.fit_transform(user_data_numeric)
    restaurant_data_imputed = imputer.fit_transform(restaurant_data_numeric)

    # 비율 계산 (빈도수를 전체 합으로 나눔)
    restaurant_data_proportions = restaurant_data_imputed / restaurant_data_imputed.sum(axis=1, keepdims=True)

    # 정규화
    scaler = StandardScaler()
    user_data_scaled = scaler.fit_transform(user_data_imputed)
    restaurant_data_scaled = scaler.fit_transform(restaurant_data_proportions)

    return user_data_scaled, restaurant_data_scaled, user_ids, restaurant_ids, restaurant_data

# 2. PCA 수행
def perform_pca(data, n_components=3):
    """
    입력 데이터를 주성분으로 변환합니다.

    Parameters:
        data (ndarray): 입력 데이터.
        n_components (int): 주성분 개수.

    Returns:
        ndarray: 축소된 데이터
    """
    pca = PCA(n_components=n_components)
    return pca.fit_transform(data)

# 3. 유사도 계산
def calculate_recommendation_scores(user_features, restaurant_features, weights=None):
    """
    사용자와 식당 데이터 간 코사인 유사도를 계산합니다.

    Parameters:
        user_features (ndarray): 사용자 데이터.
        restaurant_features (ndarray): 식당 데이터.
        weights (list): 각 열에 대한 가중치 리스트.

    Returns:
        ndarray: 유사도 점수 배열
    """
    
    if weights is not None:
        # 가중치 적용
        weights = np.array(weights)
        restaurant_features = restaurant_features * weights
    return cosine_similarity(user_features, restaurant_features)


# 4. 추천 리스트 생성
def generate_recommendations(scores, user_ids, restaurant_ids, top_n=5):
    """
    사용자별 상위 N개의 추천 식당을 생성합니다.

    Parameters:
        scores (ndarray): 유사도 점수 배열.
        user_ids (list): 사용자 ID 리스트.
        restaurant_ids (list): 식당 ID 리스트.
        top_n (int): 추천할 식당 개수.

    Returns:
        DataFrame: 추천 결과
    """
    recommendations = []
    for i, user_id in enumerate(user_ids):
        top_indices = scores[i].argsort()[::-1][:top_n]
        top_restaurants = [restaurant_ids[j] for j in top_indices]
        top_scores = [scores[i][j] for j in top_indices]
        recommendations.append({"user_id": user_id, "recommended_restaurants": top_restaurants, "scores": top_scores})
    return pd.DataFrame(recommendations)

# 5. 결과 출력 함수
def print_recommendations(recommendations):
    """
    추천 결과를 출력합니다.

    Parameters:
        recommendations (DataFrame): 추천 결과 데이터프레임
    """
    for _, row in recommendations.iterrows():
        print(f"User ID: {row['user_id']}")
        print("Recommended Restaurants and Scores:")
        for restaurant, score in zip(row['recommended_restaurants'], row['scores']):
            print(f"  - {restaurant}: {score:.4f}")
        print()
        
def main(file_path_user, file_paths_restaurant, n_components=3, top_n=5):
    """
    PCA 기반 추천 시스템 실행.

    Parameters:
        file_path_user (str): 사용자 데이터 파일 경로.
        file_paths_restaurant (list): 식당 데이터 파일 경로 리스트.
        n_components (int): PCA 주성분 개수.
        top_n (int): 추천할 식당 개수.
    """
    # 데이터 로드 및 전처리
    user_data, restaurant_data, user_ids, restaurant_ids, raw_restaurant_data = load_and_preprocess(
        file_path_user, file_paths_restaurant
    )

    # 가중치 정의 (기본 가중치 1, 특정 열에 대해 1.3 적용)
    weighted_columns = [
            "고기 질이 좋아요", "기본 안주가 좋아요", "디저트가 맛있어요", "매장이 넓어요", "사진이 잘 나와요", "술이 다양해요", "아늑해요", "양이 많아요", "음료가 맛있어요",
        "음식이 맛있어요", "음악이 좋아요", "인테리어가 멋져요", "잡내가 적어요", "재료가 신선해요", "커피가 맛있어요", "컨셉이 독특해요", "특별한 메뉴가 있어요",
        "건강한 맛이에요", "단체모임 하기 좋아요", "대화하기 좋아요", "빵이 맛있어요", "혼밥하기 좋아요", "가격이 합리적이에요", "놀거리가 많아요", "매장이 청결해요",
        "메뉴 구성이 알차요", "비싼 만큼 가치있어요", "친절해요", "테마가 재미있어요", "화장실이 깨끗해요", "맛있어요", "신선해요", "집중하기 좋아요", "반찬이 잘 나와요",
        "현지 맛에 가까워요", "뷰가 좋아요", "오래 머무르기 좋아요", "종류가 다양해요", "차분한 분위기에요", "독특한 책이 많아요", "베스트셀러가 많아요", "시설이 깔끔해요",
        "음식이 빨리 나와요", "주차하기 편해요", "직접 잘 구워줘요", "카페 같은 분위기에요", "특별한 날 가기 좋아요", "혼술하기 좋아요", "주문제작을 잘해줘요",
        "야외공간이 멋져요", "좌석이 편해요", "감각적이에요", "꽃 조합이 예뻐요", "포장이 정성스러워요", "차가 맛있어요", "코스요리가 알차요", "포장이 깔끔해요",
        "반려동물과 가기 좋아요", "라이브공연이 훌륭해요", "룸이 잘 되어있어요", "부위가 다양해요", "원산지 표시가 명확해요", "특색 있는 제품이 많아요", "식사하기 좋아요",
        "아이와 가기 좋아요", "샐러드바가 잘 되어있어요", "클럽하우스가 잘 되어있어요", "필드 상태가 좋아요", "깨끗해요", "방음이 잘돼요", "조용히 쉬기 좋아요",
        "산책로가 잘 되어있어요", "품질이 좋아요", "향신료가 강하지 않아요", "규모가 커요", "방문객이 많아요"
        ]
    weights = [1.3 if col in weighted_columns else 1 for col in raw_restaurant_data.select_dtypes(include=['float64', 'int64']).columns]

    # Debug: 가중치가 제대로 적용되었는지 확인
    print("Weights applied to columns:")
    for col, weight in zip(raw_restaurant_data.select_dtypes(include=['float64', 'int64']).columns, weights):
        print(f"{col}: {weight}")


    # 가중치 적용 (원본 데이터에 적용)
    restaurant_data_weighted = restaurant_data * weights

    # PCA 수행
    user_features = perform_pca(user_data, n_components)
    restaurant_features = perform_pca(restaurant_data_weighted, n_components)

    # 유사도 계산
    scores = calculate_recommendation_scores(user_features, restaurant_features)

    # 추천 결과 생성 및 출력
    recommendations = generate_recommendations(scores, user_ids, restaurant_ids, top_n)
    print_recommendations(recommendations)


# 실행
user_file_path = "aggregated_final_result.csv"
restaurant_file_paths = [
    "df_cafe.csv",
    "df_drink.csv",
    "df_meal.csv"
]
main(user_file_path, restaurant_file_paths, n_components=3, top_n=5)

Weights applied to columns:
index: 1
빵이 맛있어요: 1.3
친절해요: 1.3
특별한 메뉴가 있어요: 1.3
매장이 청결해요: 1.3
가성비가 좋아요: 1
커피가 맛있어요: 1.3
음료가 맛있어요: 1.3
인테리어가 멋져요: 1.3
대화하기 좋아요: 1.3
집중하기 좋아요: 1.3
건강한 맛이에요: 1.3
좌석이 편해요: 1.3
아늑해요: 1.3
디저트가 맛있어요: 1.3
음식이 맛있어요: 1.3
종류가 다양해요: 1.3
사진이 잘 나와요: 1.3
뷰가 좋아요: 1.3
혼밥하기 좋아요: 1.3
메뉴 구성이 알차요: 1.3
비싼 만큼 가치있어요: 1.3
매장이 넓어요: 1.3
화장실이 깨끗해요: 1.3
차분한 분위기에요: 1.3
컨셉이 독특해요: 1.3
혼술하기 좋아요: 1.3
아이와 가기 좋아요: 1.3
양이 많아요: 1.3
포장이 깔끔해요: 1.3
주차하기 편해요: 1.3
차가 맛있어요: 1.3
단체모임 하기 좋아요: 1.3
음식이 빨리 나와요: 1.3
재료가 신선해요: 1.3
반려동물과 가기 좋아요: 1.3
선물하기 좋아요: 1
오래 머무르기 좋아요: 1.3
특별한 날 가기 좋아요: 1.3
주문제작을 잘해줘요: 1.3
야외공간이 멋져요: 1.3
음악이 좋아요: 1.3
태그 총합: 1
술이 다양해요: 1.3
잡내가 적어요: 1.3
룸이 잘 되어있어요: 1.3
환기가 잘 돼요: 1
코스요리가 알차요: 1.3
반찬이 잘 나와요: 1.3
현지 맛에 가까워요: 1.3
고기 질이 좋아요: 1.3
샐러드바가 잘 되어있어요: 1.3
향신료가 강하지 않아요: 1.3
직접 잘 구워줘요: 1.3
기본 안주가 좋아요: 1.3
User ID: 아암마시따
Recommended Restaurants and Scores:
  - 베나레스 숙대점: 0.9881
  - 모토카레시: 0.9607
  - 더블랙웨일바: 0.9590
  - 탄막: 0.9556
  - 이웃집영준이: 0.9543

User ID: 밍닝몽
Recommended Restaurants and